In [6]:
%pip install geatpy==2.7.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install baostock

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/55.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/55.2 kB ? eta -:--:--
     ------- -------------------------------- 10.2/55.2 kB ? eta -:--:--
     ---------------------------- --------- 41.0/55.2 kB 991.0 kB/s eta 0:00:01
     ---------------------------- --------- 41.0/55.2 kB 991.0 kB/s eta 0:00:01
     ---------------------------- --------- 41.0/55.2 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 55.2/55.2 kB 221.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import baostock as bs

bs.login()

huaxia_stock = bs.query_history_k_data_plus("sz.002928",
"date,close",
start_date='2019-07-01', end_date='2023-07-31',
frequency="d", adjustflag="3")

chunqiu_stock = bs.query_history_k_data_plus("sh.601021",
"date,close",
start_date='2019-07-01', end_date='2023-07-31',
frequency="d", adjustflag="3")

jixiang_stock = bs.query_history_k_data_plus("sh.603885",
"date,close",
start_date='2019-07-01', end_date='2023-07-31',
frequency="d", adjustflag="3")

zhguo_stock = bs.query_history_k_data_plus("sh.601111",
"date,close",
start_date='2019-07-01', end_date='2023-07-31',
frequency="d", adjustflag="3")

login success!


In [11]:
import geatpy as ea
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 自定义问题类，多参数的优化问题
class MyProblem(ea.Problem): # 继承Problem父类
    def __init__(self):
        self.name = 'PairTrading' # 初始化name（函数名称，可以随意设置）
        self.Dim = 7 # 初始化Dim（决策变量维数）
        self.maxormins = 1 # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
        self.M = 1 # 初始化M（目标维数）
        self.varTypes = [1, 0, 0, 0, 0, 0, 0] # 除了第一个是代表布林带均值计算的MA的n值，其余都是0，代表连续变量
        self.lb = [1, 0.1, 0.1, 0, 0, 0, 0] # 决策变量下界
        self.ub = [100, 3, 3, 1, 1, 1, 1] # MA的n值上界为100，布林带上下标准差数量上界为3，其余都是1
        self.ReconbinOpt = 'xovdp'
        self.borders = np.array([[1] * self.Dim,[1]*self.Dim]) # 生成区域描述器，这里是两个长度为Dim的列表，列表中的元素都是1
        self.mutStyle = 'mutg'  # 使用高斯变异

    def aimFunc(self, pop):
        return super().aimFunc(pop) # 调用父类的aimFunc方法求解目标函数,pop为传入的种群对象


In [12]:
# 实例化问题对象
problem = MyProblem()
# 实例化种群对象（此时种群还没被初始化，仅仅是完成了种群对象的实例化）
Encoding = 'RI'       # 编码方式 
NIND = 100             # 种群规模
Field = ea.crtfld(Encoding, problem.varTypes, problem.borders)
population = ea.Population(Encoding, Field, NIND) # 实例化种群对象
# 调用编程模板(传入种群对象population，设置参数maxormins和problem)
algorithm = ea.soea_SEGA_templet(problem, population, MAXGEN=50, FINITPOP=100) # finitpop为初始种群规模
# 自定义交叉、变异、选择算子
algorithm.mutOper.Pm = 0.1 # 设置变异概率
algorithm.recOper.XOVR = 0.8 # 设置交叉概率
algorithm.selFunc = 'tour' # 设置选择算子

C:\Windows\Temp\ipykernel_2112\1475093848.py:6: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  Field = ea.crtfld(Encoding, problem.varTypes, problem.borders)


In [ ]:
# 运行
[population, obj_trace, var_trace] = algorithm.run() # 执行算法模板，得到最后一代种群以及进化记录器